# StaTexNet - Network Encoding Statistics for Textures

## Dependencies & Hyperparameters

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import sys
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('/home/gridsan/ckoevesdi/.local/lib/python3.9/site-packages/')
from pytorch_metric_learning import losses

sys.path.append('/home/gridsan/ckoevesdi/PooledStatisticsMetamers/poolstatmetamer/')
import utils.statnetencoder as sne
import importlib
import imp

#!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py

#sys.path.append(r'C:\Users\chris\Documents\MIT\Statistics_analysis_code\PyTorchSteerablePyramid')
sys.path.append('/home/gridsan/ckoevesdi/PyTorchSteerablePyramid/')
import steerable
import steerable.utils as utils
from steerable.SCFpyr_PyTorch import SCFpyr_PyTorch

torch.manual_seed(16)

#use GPU 2
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
#hyperparams
num_epochs = 40
batch_size = 20
crop_size = 128 #changed to 64 as fourcrop doesn't exist
num_stats = 50 
optimizer_type='adam'
#optimizer_type='sgd'
learning_rate = 0.001
num_crops = 5 #changed this to four

sparsity_penalty = 0.001

#dataset location
#dtd_folder = '/gridsan/ckoevesdi/data/dtd_torch/dt/'
dtd_folder = 'home/gridsan/ckoevesdi/data/OT/dtd_torch/dtd/'

## Define DataLoader

In [4]:
#loading_transforms = torchvision.transforms.Compose([#transforms.CenterCrop(size=300),
#                                                    #transforms.RandomRotation(degrees=180),
#                                                    transforms.Grayscale(), 
#                                                    #transforms.TenCrop(size=crop_size),
#                                                    #transforms.RandomRotation(degrees=[0,90,180,270]),
                                                    #transforms.RandomVerticalFlip(p=0.5), #Commented out
                                                    #transforms.RandomHorizontalFlip(p=0.5), #Commented out
#                                                    transforms.FiveCrop(size=crop_size), #changed back to fivecrop
                                                    #transforms.functional.vflip(),
                                                    #transforms.functional.hflip(),
 #                                                   transforms.Lambda(lambda crops: torch.stack([transforms.PILToTensor()(crop) for crop in crops])),
#                                                    transforms.ConvertImageDtype(torch.float32)])
                                                    #transforms.PILToTensor()])
loading_transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1), # Slight brightness & contrast adjustment
    transforms.FiveCrop(size=crop_size), 
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])), # Changed PILToTensor to ToTensor as it's more standard
    transforms.ConvertImageDtype(torch.float32)
])

#use training set for now
dtd_dataset = torchvision.datasets.DTD(root='/home/gridsan/ckoevesdi/data/OT/dtd_torch/', split='train', partition=10, 
                                       transform=loading_transforms, target_transform=None,
                                       download=False) #ah das datenset muss so aussehen wie es auf der website auch ist, deswegen kann man auch download false machen

sampler = data.RandomSampler(dtd_dataset)

dtd_dataloader = DataLoader(dtd_dataset, 
                            sampler=sampler,
                            batch_size=batch_size, 
                            shuffle=False)

tensor2pil_transform = transforms.ToPILImage()
print(len(dtd_dataset))

1882


In [4]:
loading_transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1), # Slight brightness & contrast adjustment
    transforms.FiveCrop(size=crop_size), 
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])), # Changed PILToTensor to ToTensor as it's more standard
    transforms.ConvertImageDtype(torch.float32)
])

#use training set for now
dtd_dataset = torchvision.datasets.DTD(root='/home/gridsan/ckoevesdi/data/OT/dtd_torch/', split='train', partition=1, 
                                       transform=loading_transforms, target_transform=None,
                                       download=False) #ah das datenset muss so aussehen wie es auf der website auch ist, deswegen kann man auch download false machen


dtd_dataloader = DataLoader(dtd_dataset, 
                            batch_size=batch_size, 
                            shuffle=False)

tensor2pil_transform = transforms.ToPILImage()
print(len(dtd_dataset))

1879


## Test Dataloader

In [ ]:
if(True):
    for n, texture_batch in enumerate(dtd_dataloader):
         #grab texture batch and generate matching labels
         output = texture_batch[0].to(device)
         output = torch.flatten(output, start_dim=0, end_dim=1)
         texture_labels = torch.repeat_interleave(torch.arange(batch_size),num_crops)
         #apply random permutation
         print(texture_labels)
         #loop through batch and plot images
         for j in range(batch_size):
             plt.figure(figsize=(8,4))
             for i in range(num_crops):
                 plt.subplot(2,5,i+1)
                 plt.imshow(tensor2pil_transform(output[i+j*num_crops,:,:,:]))
                 plt.axis('off')
             plt.show()
         if(n==1):
             break;
    
#tensor2pil_transform(output[4,:,:,:])

In [ ]:
# Create an iterator from the DataLoader
dataloader_iterator2 = iter(dtd_dataloader)

# Get the first batch
first_batch = next(dataloader_iterator2)

# Extract data and label from the first batch
data, label = first_batch
first_sample, first_label = data[0], label[0]
# Assuming x is your (1, 128, 128) shaped matrix
#plt.imshow(first_sample[0,0, :, :], cmap='gray'); plt.show()

first_sample = first_sample.to(device)
import matplotlib.pyplot as plt



print(first_sample[:,:, :, :].shape)
print(first_label)
x = statnet_model.encoder(first_sample[1,:, :, :])
print(x.shape)

## Define Model & Optimizer

In [ ]:
importlib.reload(sne)
statnet_model = sne.StatNetEncoder(img_size=(crop_size,crop_size),
                                   batch_size=batch_size,
                                   num_stats=num_stats,
                                   device=device)
statnet_model.to(device)
stat_labels = statnet_model.getsstatlabels(device)
#optimizer
if(optimizer_type=='sgd'):
    optimizer = torch.optim.SGD(statnet_model.parameters(), lr=learning_rate)#, momentum=learning_momentum)
elif(optimizer_type=='adam'):
    optimizer = torch.optim.Adam(statnet_model.parameters(), lr=learning_rate)
elif(optimizer_type=='adagrad'):
    optimizer = torch.optim.Adagrad(statnet_model.parameters(), lr=learning_rate)
elif(optimizer_type=='adadelta'):
    optimizer = torch.optim.Adadelta(statnet_model.parameters(), lr=learning_rate)
else:
    print('No Optimizer Specified! Adam is default!')
    optimizer = torch.optim.Adam(statnet_model.parameters(), lr=learning_rate)

In [ ]:
print('Total Stats:',len(stat_labels))
summation=0
for stat in ['edge_mean','edge_variance','edge_stop','edge_correlation','phase_correlation','mean','variance','bandpass_variance','skew','kurtosis']:
    num_this_stat = len([s for s in stat_labels if s.weight_category==stat])
    print(f'{stat} Stat:',num_this_stat)
    summation += num_this_stat
print('Stats Sum to:',summation)
#[s for s in stat_labels] #print all stats with details

## Run Training

In [ ]:
loss_func_contrastive = losses.GeneralizedLiftedStructureLoss()
training_loss = []
training_representation_loss = []
training_sparsity_loss = []
num_total_epochs = 0
#[s for s in stat_labels] #print all stats with details
statnet_model.train() # Set model to training mode
optimizer.zero_grad()
statnet_model.zero_grad()

In [ ]:
import time
start_time = time.time()
print('Initial Weights:', statnet_model._w)

In [ ]:
print('Starting Training:')
for i, epoch in enumerate(range(num_epochs)):
    for j, texture_batch in enumerate(dtd_dataloader):
        #grab texture batch and generate matching labels
        output = texture_batch[0].to(device)
        #print(output.shape)
        output = torch.flatten(output, start_dim=0, end_dim=1)
        texture_labels = torch.repeat_interleave(torch.arange(batch_size),num_crops)
        perm = torch.randperm(batch_size * num_crops)

        current_batch_size = output.shape[0] // num_crops
        
        if current_batch_size < batch_size:
            continue  # Skip this batch if it's smaller than the usual batch size

        output = output[perm]
        texture_labels = texture_labels[perm]
        #print(texture_labels.shape)
        #calculate stats
        stats_vector = statnet_model.forward(output) #statsvector torch.Size([batchsize*crops, 50])
        #print(texture_labels.shape)
        print(stats_vector.shape)
        
        #loss definitions
        representation_loss = loss_func_contrastive(stats_vector, texture_labels)
        sparsity_loss = statnet_model.sparse_loss()
        loss = representation_loss + (sparsity_penalty * sparsity_loss) 
        #loss = triplet_loss(stats_vector, texture_labels, texture_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        #optimizer.step()
        #optimizer.zero_grad()
        #loss.backward()

        print('*',end='')
        #training_loss.append(loss.item())      
        if(j%30==0):
            print(loss.item())
        if(j==100):
            break;
        training_representation_loss.append(representation_loss.item())
        training_sparsity_loss.append(loss.item())    
        training_loss.append(loss.item())
    num_total_epochs = num_total_epochs + 1
    print(f'Finished Epoch {i}. Loss at {loss}.')
    #print('Initial Weights:', statnet_model._w)    
    if(i%10==0):
        compressor_mat = statnet_model.w.T.data.cpu().numpy()
        plt.imshow(compressor_mat)
        plt.colorbar() 
        plt.show()
print('All Done!')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
plt.plot(training_loss)

In [ ]:
compressor_mat = statnet_model.w.T.data.cpu().numpy()
print(compressor_mat.shape)
plt.figure(figsize=(10,5))
plt.imshow(compressor_mat)
plt.colorbar()
print(compressor_mat)

In [ ]:
stat_sums = np.sum(np.abs(compressor_mat),axis=0) #how weighted is each of the 150 stats?
stat_index_array = np.argsort(stat_sums)[::-1] #get their importance order

ordered_stat_sums = stat_sums[stat_index_array]
ordered_stat_labels = np.array(stat_labels,dtype=object)[stat_index_array]
ordered_stats_labels_pretty = [' '.join([''.join(str(item)) for item in row if item not in ['',None]]) for row in ordered_stat_labels]
n=25
print(f'Most Important {n} Stats:')
for i in range(n):
    print(ordered_stat_sums[i],ordered_stats_labels_pretty[i])

#plot the most important stats for contrastive learning
fig, ax = plt.subplots()
im = ax.imshow(compressor_mat[:,stat_index_array[:n]].T, aspect='auto')
fig.colorbar(im)
# Show all ticks and label them with the respective list entries
top_stats = ordered_stats_labels_pretty[:n]
ax.set_yticks(np.arange(len(top_stats)), labels=top_stats)
#Rotate the tick labels and set their alignment.
#plt.setp(ax.get_yticklabels(), rotation=45, ha="right",
#         rotation_mode="anchor")
plt.tight_layout()
plt.show()    
    
#plot most important stats for contrastive learning
#plt.imshow(compressor_mat[:,stat_index_array[:n]].T)

print(f'Least Important {n} Stats:')
for i in range(1,n+1):
    print(ordered_stat_sums[-i],ordered_stats_labels_pretty[-i])
    
    
#plot the most important stats for contrastive learning
fig, ax = plt.subplots()
im = ax.imshow(compressor_mat[:,stat_index_array[-n:]].T, aspect='auto')
fig.colorbar(im)
# Show all ticks and label them with the respective list entries
bot_stats = ordered_stats_labels_pretty[-n:][::-1]
ax.set_yticks(np.arange(len(bot_stats)), labels=bot_stats)
#Rotate the tick labels and set their alignment.
#plt.setp(ax.get_yticklabels(), rotation=45, ha="right",
#         rotation_mode="anchor")
plt.tight_layout()
plt.show()    

In [ ]:
?list.insert
print(output.shape)

print(texture_label_list.shape)

In [ ]:
X_cpu = X.detach().cpu().numpy()
print(np.any(np.isnan(X_cpu)))
print(np.any(np.isinf(X_cpu)))
print(X_cpu)

In [ ]:
from tsne_torch import TorchTSNE as TSNE

# 1. Get model embeddings
stats_vector = statnet_model.forward(output)
X = stats_vector  # Assuming shape (n_samples, d)

# 2. Compute t-SNE embeddings
X_emb = TSNE(n_components=2, perplexity=8, n_iter=10000, verbose=True).fit_transform(X)

# 3. Adjust texture labels
texture_label = texture_labels[:-5]
texture_label_list = texture_label.numpy()

# 4. Get unique textures
unique_textures = set(texture_label_list)

# 5. Plot t-SNE embeddings
plt.figure(figsize=(10, 10))
for texture in unique_textures:
    texture_index = np.where(texture_label_list == texture)[0]
    plt.scatter(X_emb[texture_index, 0], X_emb[texture_index, 1], label=str(texture))

# Optional: If you have too many labels, the legend might become cluttered.
# If you have fewer labels, uncomment the next line.
# plt.legend(loc='upper right')

plt.title(f'T-sne embeddings: {num_crops} crops, {batch_size} textures')
plt.tight_layout()
plt.show()



In [ ]:
print(stats_vector.shape)
print(X_emb)
#print(unique_textures)

In [ ]:
unique_textures = set(texture_label_list)

plt.figure(figsize=(10, 10))
for texture in unique_textures:
    texture_index = np.where(texture_label_list == texture)[0]
    plt.scatter(X_emb[texture_index, 0], X_emb[texture_index, 1], label=str(texture))
    
    # Optional: Annotate the centroid of each texture cluster instead of all points
    centroid = X_emb[texture_index].mean(axis=0)
    plt.annotate(str(texture), (centroid[0], centroid[1]))

plt.legend()
plt.title(f'T-sne embeddings: {num_crops} crops, {batch_size} textures')
plt.tight_layout()
plt.show()
